I was inspired by this meme that assigns the 7 colors of the lesbian pride flag to letters of the word "lesbian." I wanted to see what other words I could create with this pattern and how I could experiment with web accessibility features to make this more into nonsensical spoken 
poetry

<img src="meme.jpg" height="300px"/>

I found a plaintext list of English words online so I could filter to see all valid English words that can be spelled with the letters in "lesbian"

In [2]:
words = open("words.txt").read().splitlines()
print(len(words))

45402


In [3]:
gayLetters = list("lesbian")
isGayWord = lambda word: all(letter in gayLetters for letter in word.lower())
print(isGayWord("Bean"))
print(isGayWord("lemon"))

True
False


In [4]:
gayWords = list(filter(isGayWord, words))
print(gayWords)

['Ababa', 'abase', 'abases', 'Abba', 'abbe', 'Abe', 'Abel', 'Abelian', 'Abilene', 'able', 'Aeneas', 'ail', 'Aileen', 'aisle', 'Al', 'Alan', 'alas', 'alba', 'Albania', 'Albanian', 'Albanians', 'ale', 'alee', 'Ali', 'alias', 'aliases', 'alibi', 'alibis', 'alien', 'aliens', 'all', 'Allan', 'allele', 'alleles', 'Allen', 'allies', 'Allis', 'an', 'Anabel', 'anal', 'aniline', 'anise', 'Ann', 'Anna', 'annal', 'annals', 'Anne', 'Annie', 'ANSI', 'as', 'Asia', 'Asian', 'Asians', 'asinine', 'ass', 'assail', 'assails', 'assassin', 'assassins', 'asses', 'assess', 'assesses', 'babble', 'babbles', 'babe', 'Babel', 'babes', 'babies', 'bail', 'bale', 'bales', 'Bali', 'Balinese', 'ball', 'balls', 'balsa', 'ban', 'banal', 'banana', 'bananas', 'bane', 'bans', 'basal', 'base', 'baseball', 'baseballs', 'Basel', 'baseless', 'baseline', 'baselines', 'baseness', 'bases', 'Basie', 'basil', 'basin', 'basins', 'basis', 'bass', 'basses', 'be', 'bean', 'beans', 'bee', 'Beebe', 'been', 'bees', 'Bela', 'belie', 'belie

This is more words than I expected! A fair amount of them are names or other proper nouns. It tickles me to know that the name Isabel is in that list, which is my legal name

In [5]:
# thank you Allison for these functions!
from IPython.display import display, HTML
def show_html(src):
    return display(HTML(src), metadata=dict(isolated=True))
    
def mkdiv(content, **kwargs):
    if 'position' not in kwargs:
        kwargs['position'] = 'absolute'
    style_str = ' '.join([": ".join((k.replace('_', '-'), v))+";" for k, v in kwargs.items()])
    return f"<div style='{style_str}'>{content}</div>"

In [6]:
letterToColor = {
    "l": "#D52D00",
    "e": "#EF7627",
    "s": "#FF9A56",
    "b": "#FFFFFF",
    "i": "#D162A4",
    "a": "#B55690",
    "n": "#A30262",
}

# render a single word as flag stripes + a text label using flexbox
# ...because i love flexbox
def wordToFlag(word):
    children = []
    for letter in word:
        color = letterToColor[letter]
        newDiv = mkdiv("", 
                       position="relative",
                       height="10px", 
                       width="120px",
                       background_color=color)
        children.append(newDiv)

    label = f"<p style='text-align: center; margin: 4px 0 0 0'>{word}</p>"
    children.append(label)
    return mkdiv("".join(children), 
                 position="relative",
                 display="flex",
                 flex_direction="column",
                 align_items="center")

In [7]:
show_html(wordToFlag("lesbian"))

In [8]:
# render a list of words as a grid of evenly spaced flags
def wordsToFlagGrid(words):
    children = [wordToFlag(word.lower()) for word in words]
    content = mkdiv("".join(children),
                width="100%",
                display="flex",
                flex_wrap="wrap",
                align_items="flex-end",
                justify_content="center",
                gap="24px",
                padding_bottom="24px")

    # the white stripe is invisible on default white bg
    return f"""
    <style>
        body {{
            background-color: #bdbdbd !important;
        }}
    </style>
    {content}
    """

In [9]:
show_html(wordsToFlagGrid(["lesbian", "les", "bian", "AAAAAAAAA"]))

In [36]:
import random

randomGayWords = random.sample(gayWords, 25)
flagGrid = wordsToFlagGrid(["lesbian", *randomGayWords])
show_html(flagGrid)

It's interesting to see the variation in word length here, these almost feel like bar graphs for number of letters in a word. The longest words I've seen are "silliness" and "senselessness". Also of course lots of pluralized nouns since we have the "s". The list of valid "lesbian" words above is essentially half the size if appears to be since almost every noun is counted twice after it's pluralized. I see some words that are missing from the plaintext word list though. For example, we have "babes" but not "babe."

One feature that adds to the meme's humor is that nonsensical words are included, like "sssss" or "bbibbi". I wanted to generate some of these random syllables to mix in with valid words. I wanted pronounceable syllables that obey English phonotactic rules. The below code is definitely overkill for how few letters/phonemes are in the word "lesbian", but I was a bit of a syllable nerd in my linguistics classes.

English syllable structure is COMPLEX compared to other languages (/strength/ = CCCVCCC? gross.) so this is a hard problem to solve programmatically.

In [37]:
#https://en.wikipedia.org/wiki/Sonority_hierarchy

sonorityScale = {
    0: ["b"],           # stops
    1: ["s"],           # fricatives
    2: ["n"],           # nasals
    3: ["l"],           # laterals
    4: ["a", "e", "i"], # vowels
}

# lol this pattern breaks down so quickly if i add more phonemes but oh well
sonorityTransitions = {
    0: [3, 4],
    1: [2, 3, 4],
    2: [4],
    3: [4],
    4: [],
}

# i thought i would come back later to implement syllable codas but i Did Not
def randomSyllable(coda=False):
    syllable = ""
    i = random.randint(0, 4)
    while i < 4:
        syllable += random.choice(sonorityScale[i])
        i = random.choice(sonorityTransitions[i])
    syllable += random.choice(sonorityScale[4])
    return syllable

In [43]:
def getRandomSyllables(n, minSyl=2, maxSyl=10):
    randomSyllables = []
    for i in range(n):
        randomDup = random.randint(minSyl, maxSyl)
        syllable = randomSyllable()
        randomSyllables.append(syllable * randomDup)
    return randomSyllables
    
flagGrid = wordsToFlagGrid(getRandomSyllables(10))
show_html(flagGrid)

In [38]:
open("syllables3.html", "w").write(flagGrid)

17622

This is just screaming to be fed through my browser's default text to speech feature. I had a hard time capturing a video with sound to include here, but it's hilarious.

How do the fake syllable words look alongside real words? From looking at the stripes it's pretty clear which ones have repeated characters and which don't. This output is even funnier to run through text to speech because the random sounds in between real words really confuses the intonation

In [47]:
wordMix = [*getRandomSyllables(12, 2, 4), *random.sample(gayWords, 12)]
random.shuffle(wordMix)
flagGrid = wordsToFlagGrid(wordMix)
show_html(flagGrid)

In [34]:
open("syllables4.html", "w").write(flagGrid)

16083

Working with text to speech was unexpectedly hilarious, so let's try removing the text labels and instead adding screen-reader friendly HTML attributes to the flag stripes. My goal is for a screen reader to be able to "speak" the flag and have it pronounced the same as the text label, but with no text rendering.

In [138]:
def mkspan(content, **kwargs):
    if 'position' not in kwargs:
        kwargs['position'] = 'absolute'
    style_str = ' '.join([": ".join((k.replace('_', '-'), v))+";" for k, v in kwargs.items()])
    return f"<button aria-description='{content}' style='{style_str}'></button>"

def mkdiv2(content, title, **kwargs):
    if 'position' not in kwargs:
        kwargs['position'] = 'absolute'
    style_str = ' '.join([": ".join((k.replace('_', '-'), v))+";" for k, v in kwargs.items()])
    return f"<div contenteditable='true' title='{title}' style='{style_str}'>{content}</div>"

In [139]:
def wordToFlag2(word):
    children = []
    for letter in word:
        color = letterToColor[letter]
        span = mkspan(letter, 
                       position="relative",
                       height="10px", 
                       width="120px",
                       background_color=color)
        children.append(span)

    return mkdiv2("".join(children),
                  word,
                 position="relative",
                 display="flex",
                 flex_direction="column",
                 align_items="center")

flag = wordToFlag2("lesbian")
print(flag)
show_html(flag)
open("screenReaderTest.html", "w").write(flag)

<div contenteditable='true' title='lesbian' style='position: relative; display: flex; flex-direction: column; align-items: center;'><button aria-description='l' style='position: relative; height: 10px; width: 120px; background-color: #D52D00;'></button><button aria-description='e' style='position: relative; height: 10px; width: 120px; background-color: #EF7627;'></button><button aria-description='s' style='position: relative; height: 10px; width: 120px; background-color: #FF9A56;'></button><button aria-description='b' style='position: relative; height: 10px; width: 120px; background-color: #FFFFFF;'></button><button aria-description='i' style='position: relative; height: 10px; width: 120px; background-color: #D162A4;'></button><button aria-description='a' style='position: relative; height: 10px; width: 120px; background-color: #B55690;'></button><button aria-description='n' style='position: relative; height: 10px; width: 120px; background-color: #A30262;'></button></div>


985

In [152]:
def wordsToFlagGrid2(words):
    children = [wordToFlag2(word.lower()) for word in words]
    content = mkdiv("".join(children),
                width="100%",
                display="flex",
                flex_wrap="wrap",
                align_items="flex-end",
                justify_content="center",
                gap="24px",
                padding_bottom="24px")

    # the white stripe is invisible on default white bg
    return f"""
    <style>
        body {{
            background-color: #bdbdbd !important;
        }}

        button {{
            border: unset;
        }}
    </style>
    {content}
    """

flags = wordsToFlagGrid2("i sense lesbian silliness".split(" "))
show_html(flags)
open("screenReaderTest.html", "w").write(flags)

3538

I tried a few hacks but this is proving to be more difficult than I thought. I tried the Chrome text to speech feature and MacOS's screen reader, but by putting letters in separate div, p, span, etc. tags they will be read as separate letters, not strung together into words. I guess that makes sense. I tried assigning the `aria-label` or `title` attribute to the flag's parent div but that didn't work either, nothing was spoken aloud.

https://css-tricks.com/almanac/properties/s/speak/ was an interesting read, I haven't seen those CSS properties before

In [15]:
flagGrid = wordsToFlagGrid("i sense lesbian silliness".split(" "))
show_html(flagGrid)